In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('LinearRegression').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
training = spark.read.format('libsvm').load('../../SparkData/sample_linear_regression_data.txt')

In [6]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

Feature column has a vector of features that belong to that row. Will need data transformations for future cases but for this version, the dataset is tailored for ease of use.

In [7]:
lr = LinearRegression(featuresCol='features', labelCol='label', 
                      predictionCol='prediction')

In [8]:
lr_model = lr.fit(training)

In [9]:
lr_model.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [10]:
lr_model.intercept

0.14228558260358093

In [11]:
training_summary = lr_model.summary

In [12]:
training_summary.r2

0.027839179518600154

In [13]:
training_summary.rootMeanSquaredError

10.16309157133015

***

In [15]:
train_data, test_data = training.randomSplit([0.7, 0.3])

In [16]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                354|
|   mean| 0.4374803944108104|
| stddev| 10.737167735074655|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [17]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                147|
|   mean|-0.1780051680390278|
| stddev|   9.25061554714721|
|    min|-23.487440120936512|
|    max|  22.31738046492344|
+-------+-------------------+



In [18]:
better_model = lr.fit(train_data)

In [19]:
test_results = better_model.evaluate(test_data)

In [20]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-21.572860504174656|
|-23.344282244886077|
|-20.638781769820717|
|-17.492535305177018|
| -19.01399572834542|
|-18.737730026940216|
| -13.64195930500222|
|-15.388465485101463|
|-16.333124843127084|
|-12.341754200758494|
|-15.881776712696453|
| -9.844310812858298|
|  -15.1352096472854|
|-13.148151763547261|
| -9.195501512804494|
|-10.703514762404698|
|-13.613177629969165|
| -6.244008045125808|
|-10.073476848502155|
|-10.379811653911212|
+-------------------+
only showing top 20 rows



In [21]:
test_results.meanSquaredError

87.51198251983733

***

In [22]:
unlabeled_data = test_data.select('features')

In [23]:
# only returns the unlabeled feature columns
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [24]:
predictions = better_model.transform(unlabeled_data)

In [25]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -1.9145796167618572|
|(10,[0,1,2,3,4,5,...|  1.9118944807202722|
|(10,[0,1,2,3,4,5,...|  0.8560189802061808|
|(10,[0,1,2,3,4,5,...|-0.00166505170632...|
|(10,[0,1,2,3,4,5,...|  2.3217887070343157|
|(10,[0,1,2,3,4,5,...|  2.4762997514874843|
|(10,[0,1,2,3,4,5,...| -2.5093900462748917|
|(10,[0,1,2,3,4,5,...| -0.6971935559200266|
|(10,[0,1,2,3,4,5,...|  1.2766418685846495|
|(10,[0,1,2,3,4,5,...| -0.8115814056070356|
|(10,[0,1,2,3,4,5,...|   4.003609212729612|
|(10,[0,1,2,3,4,5,...|  -2.013039552571127|
|(10,[0,1,2,3,4,5,...|  3.7034072817449406|
|(10,[0,1,2,3,4,5,...|  2.5480214216382295|
|(10,[0,1,2,3,4,5,...| -1.3821967914387268|
|(10,[0,1,2,3,4,5,...| 0.21435763903180044|
|(10,[0,1,2,3,4,5,...|   3.257259029594484|
|(10,[0,1,2,3,4,5,...| -4.0497059955301165|
|(10,[0,1,2,3,4,5,...|-0.11468052668895584|
|(10,[0,1,2,3,4,5,...|  0.571424